In [1]:
import pandas as pd
import astropy
# import bilby
import numpy as np
from matplotlib import pyplot as plt
import healpy as hp
from ligo.skymap import io, postprocess, plot, bayestar
from glob import glob
from scipy.stats import norm
from astropy import units as u
from astroquery.vizier import Vizier
from scipy.special import gammaincinv

/Users/kruthik/opt/miniconda3/envs/ghosh_sky_tiling/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from astropy.cosmology import WMAP9 as cosmo
from astropy.table import Column
import astropy.units as u
import astropy.constants as c

filename = "18951_bayestar.fits.gz"
prob, distmu, distsigma, distnorm = hp.read_map(filename, field=range(4))
npix = len(prob)
nside = hp.npix2nside(npix)
pixarea = hp.nside2pixarea(nside)

Vizier.ROW_LIMIT = -1
cat, = Vizier.get_catalogs('J/ApJS/199/26/table3')
completeness = 0.5
alpha = -1.0
MK_star = -23.55
MK_max = MK_star + 2.5*np.log10(gammaincinv(alpha+2, completeness))

z=(u.Quantity(cat['cz'])/c.c).to(u.dimensionless_unscaled)
MK = cat['Ktmag'] - cosmo.distmod(z)
keep = (z>0) & (MK < MK_max)
cat = cat[keep]
z = z[keep]
r = cosmo.luminosity_distance(z).to('Mpc').value
theta = 0.5 * np.pi - cat['DEJ2000'].to('rad').value
phi = cat['RAJ2000'].to('rad').value
ipix = hp.ang2pix(nside, theta, phi)
dp_dV = prob[ipix] * distnorm[ipix] * norm(distmu[ipix], distsigma[ipix]).pdf(r)/pixarea

top50 = cat[np.flipud(np.argsort(dp_dV))][:50]
top50['RAJ2000', 'DEJ2000', 'Ktmag']


/Users/kruthik/opt/miniconda3/envs/ghosh_sky_tiling/lib/python3.9/site-packages/astropy/cosmology/flrw/base.py:1130: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return quad(self._inv_efunc_scalar, z1, z2, args=self._inv_efunc_scalar_args)[0]
/Users/kruthik/opt/miniconda3/envs/ghosh_sky_tiling/lib/python3.9/site-packages/numpy/lib/function_base.py:2412: RuntimeWarning: invalid value encountered in _integral_comoving_distance_z1z2_scalar (vectorized)
  outputs = ufunc(*inputs)
/Users/kruthik/opt/miniconda3/envs/ghosh_sky_tiling/lib/python3.9/site-packages/astropy/cosmology/flrw/base.py:1341: RuntimeWarning: divide by zero encountered in log10
  val = 5.0 * np.log10(abs(self.luminosity_distance(z).value)) + 25.0


RAJ2000,DEJ2000,Ktmag
deg,deg,mag
float64,float64,float32
344.01190,36.36136,8.772
343.81122,36.67177,9.958
137.19089,-38.60788,9.566
334.86545,29.39581,9.835
359.81589,46.88923,9.307
0.00695,47.27456,9.499
343.35303,37.17527,10.151
137.01408,-37.56026,10.388


In [3]:
dp_dV

array([3.51770336e-24, 1.93074147e-29, 0.00000000e+00, ...,
       3.21178328e-30, 0.00000000e+00, 8.18695908e-31])